In [1]:
#%pip install llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

d:\Anaconda\envs\kanzlai2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#model_id = "meta-llama/Llama-3.2-1B"
model_id = "meta-llama/Llama-3.2-1B-Instruct"


Data Ingestion

In [4]:
reader = SimpleDirectoryReader(input_files=["D:/LLama3/documents/test_3.txt", "D:/LLama3/documents/test_2.txt"])
documents = reader.load_data()
len(documents)
documents[0].metadata
documents[0]
    

Document(id_='53823e08-85b3-4774-ae11-725671237235', embedding=None, metadata={'file_path': 'D:\\LLama3\\documents\\test_3.txt', 'file_name': 'test_3.txt', 'file_type': 'text/plain', 'file_size': 199, 'creation_date': '2024-10-24', 'last_modified_date': '2024-10-24'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Wenn du mit der Superbahn zu spät bist, dann kannst du 312 Monate Haft und eine sehr hohe Geldstrafe erhalten.\r\nDu wirst ebenfalls in einen Hochsicherheitstrakt gebracht und musst nur Paprika essen.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

Chunking

In [5]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)
len(nodes)
nodes[0].metadata

Parsing nodes: 100%|██████████| 2/2 [00:00<00:00, 666.19it/s]


{'file_path': 'D:\\LLama3\\documents\\test_3.txt',
 'file_name': 'test_3.txt',
 'file_type': 'text/plain',
 'file_size': 199,
 'creation_date': '2024-10-24',
 'last_modified_date': '2024-10-24'}

Embedding model (modell für embeddings verwendet, z.b. vektorisieren der daten)

In [6]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


Define LLM Model

In [7]:
llm = Groq(model="llama-3.2-1b-preview", api_key="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")

In [8]:
#from transformers import AutoTokenizer, AutoModelForCausalLM
#import torch
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#modelllm = AutoModelForCausalLM.from_pretrained(model_id)

In [9]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [10]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Generating embeddings: 100%|██████████| 2/2 [00:00<00:00, 45.45it/s]


Persist/Save Index


In [11]:
vector_index.storage_context.persist(persist_dir="D:/LLama3/vector/storage_mini")


Define Storage Context


In [12]:
storage_context = StorageContext.from_defaults(persist_dir="D:/LLama3/vector/storage_mini")


Load Index


In [13]:
index = load_index_from_storage(storage_context, service_context=service_context)


In [14]:
query_engine = index.as_query_engine(service_context=service_context)

In [15]:
query = "Was passiert, wenn ich mit meiner Superbahn zu spät dran bin?"
resp = query_engine.query(query)

In [16]:
print(resp.response)

Wenn du mit deiner Superbahn zu spät bist, wirst du in einen Hochsicherheitstrakt gebracht und musst nur Paprika essen.


In [ ]:
%pip install ollama


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ollama


In [ ]:
llm = Ollama(model="llama3.2:3b-instruct-q8_0", request_timeout=120.0)